In [ ]:
import math as math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading the data from the excel file
df = pd.read_excel("newspaper problem instance.xlsx")
drivers = 4
coordinates = []
for i in range(len(df)):
    coordinates.append((df.iloc[i]["xcoord"], df.iloc[i]["ycoord"]))
coordinates = np.array(coordinates)
num_coordinates = len(coordinates)

# Define the distance matrix to define the distance between two points + locations
distance_matrix = {}
for i in range(num_coordinates):
    for j in range(num_coordinates):
        dx = coordinates[i][0] - coordinates[j][0]
        dy = coordinates[i][1] - coordinates[j][1]
        distance_matrix[(i, j)] = float((dx*dx + dy*dy) ** 0.5)  # Float to print it better

depot = coordinates[0]
# index of the depot in the coordinates array (used for distance_matrix keys)
depot_index = []

# Available stops (excluding depot)
num_stops = num_coordinates - 1  # Minus 1 for the depot

# Check that the stops can be evenly divided among drivers
if num_stops % drivers != 0:
    raise ValueError(
        f"Your instance has {num_stops} stops (excl. depot), which is not evenly divisible by {drivers}. "
        "Fix your data or adjust drivers/stops_per_driver, thanks in advance."
    )

# integer number of stops each driver should handle
stops_per_driver = num_stops // drivers # To discard the decimal/fractional part
# Define the tour length, starting at the depot and the delivery boys delivering the last newspaper and not returning to depot

def tour_length_with_depot(tour):
    if not tour: # control if the tour is empty
        return 0 # if the tour is empty, the length is 0
    # tour should be a sequence of integer indices referring to rows in `coordinates`
    L = distance_matrix[(depot_index, tour[0])]  # from depot to first stop
    # add distances between subsequent stops
    for a, b in zip(tour[:-1], tour[1:]):
        L += distance_matrix[(a, b)]
        """"
        # Iterates over all consecutive pairs in the tour, such as (location0, location1), (location1, location2), 
        # tour[:-1] includes all stops except the last, tour[1:] includes all stops except the first.
        # zip combines these into pairs of consecutive locations, like couples!

        """
    return L


print(distance_matrix)
distance_matrix[(0, 2)]

{(0, 0): 0.0, (0, 1): 363.8529922922168, (0, 2): 443.8423143414787, (0, 3): 328.19658742893716, (0, 4): 333.2221481234403, (0, 5): 389.4663014947506, (0, 6): 339.8249549400397, (0, 7): 331.748398639692, (0, 8): 313.7722103692422, (0, 9): 301.3171087077533, (0, 10): 290.1103238424996, (0, 11): 293.318939040765, (0, 12): 289.21445330411825, (0, 13): 270.0018518455012, (0, 14): 290.5942188000305, (0, 15): 267.4341040331244, (0, 16): 274.6069918993324, (0, 17): 259.7941492797711, (0, 18): 413.88524979757375, (0, 19): 266.0845730214362, (0, 20): 268.47904946196456, (0, 21): 370.74249823833253, (0, 22): 244.51789300580847, (0, 23): 241.6298822579691, (0, 24): 232.95707759155977, (0, 25): 229.2640399190418, (0, 26): 282.8798331447472, (0, 27): 217.52471123989568, (0, 28): 221.98423367437607, (0, 29): 228.60008748904713, (0, 30): 210.00238093888365, (0, 31): 209.3800372528384, (0, 32): 199.9124808509964, (0, 33): 224.75764725588314, (0, 34): 196.25493624365222, (0, 35): 202.78066969018522, (0,

443.8423143414787